# Predicting wild fires in California based on Environmental Conditions 🌲🔥

Dataset used: https://www.openml.org/search?type=data&status=active&id=43606

### 1. Importing dependencies

In [ ]:
import pandas as pd
from math import pi
import re
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

### 2. Visualizing data

Loading data:

In [ ]:
# Read the ARFF file manually
with open("./california-environmental-conditions-dataset.arff", 'r') as f:
    lines = f.readlines()

# Find where @DATA starts
data_start = 0
for i, line in enumerate(lines):
    if line.strip().upper() == '@DATA':
        data_start = i + 1
        break

# Extract column names from @ATTRIBUTE lines
columns = []
for line in lines:
    if line.strip().startswith('@ATTRIBUTE'):
        # Extract attribute name (handle quoted names and names with parentheses)
        match = re.search(r'@ATTRIBUTE\s+(".*?"|\S+)\s+', line, re.IGNORECASE)
        if match:
            col_name = match.group(1).strip('"')
            columns.append(col_name)

# Read data section into DataFrame
data_lines = [line.strip() for line in lines[data_start:] if line.strip()]

# Define data types for each column
dtype_dict = {
    'Stn_Id': 'int64',
    'Stn_Name': 'str',
    'CIMIS_Region': 'str',
    'Date': 'str',
    'ETo_(in)': 'float64',
    'Precip_(in)': 'float64',
    'Sol_Rad_(Ly/day)': 'float64',
    'Avg_Vap_Pres_(mBars)': 'float64',
    'Max_Air_Temp_(F)': 'float64',
    'Min_Air_Temp_(F)': 'float64',
    'Avg_Air_Temp_(F)': 'float64',
    'Max_Rel_Hum_(%)': 'float64',
    'Min_Rel_Hum_(%)': 'float64',
    'Avg_Rel_Hum_(%)': 'float64',
    'Dew_Point_(F)': 'float64',
    'Avg_Wind_Speed_(mph)': 'float64',
    'Wind_Run_(miles)': 'float64',
    'Avg_Soil_Temp_(F)': 'float64',
    'Target': 'int64'
}

# Parse CSV-like data (handling quotes and missing values marked as '?')
data_str = '\n'.join(data_lines)
df = pd.read_csv(
    StringIO(data_str),
    names=columns,
    quotechar="'",
    escapechar='\\',
    dtype=dtype_dict,na_values='?',
    low_memory=False
    )

print(f"Data loaded successfully: {df.shape[0]} rows, {df.shape[1]} columns")
df.head()

Data insights/visualization:

In [ ]:
# Human-readable descriptions based on dataset metadata
description_map = {
    'Target': 'Binary target indicating whether a fire occurred (0 = no, 1 = yes)',

    'Stn_Id': 'CIMIS station ID (numeric identifier for weather station)',
    'Stn_Name': 'Weather station name',
    'CIMIS_Region': 'CIMIS region name',

    'Date': 'Observation date (MM/DD/YYYY)',

    'ETo_(in)': 'Reference evapotranspiration in inches',
    'Precip_(in)': 'Precipitation in inches',

    'Sol_Rad_(Ly/day)': 'Solar radiation (Ly/day)',

    'Avg_Vap_Pres_(mBars)': 'Average vapor pressure in mBars',

    'Max_Air_Temp_(F)': 'Maximum air temperature (°F)',
    'Min_Air_Temp_(F)': 'Minimum air temperature (°F)',
    'Avg_Air_Temp_(F)': 'Average air temperature (°F)',

    'Max_Rel_Hum_(%)': 'Maximum relative humidity (%)',
    'Min_Rel_Hum_(%)': 'Minimum relative humidity (%)',
    'Avg_Rel_Hum_(%)': 'Average relative humidity (%)',

    'Dew_Point_(F)': 'Dew point temperature (°F)',
    'Avg_Wind_Speed_(mph)': 'Average wind speed (mph)',
    'Wind_Run_(miles)': 'Wind run (miles) — cumulative distance traveled by wind',

    'Avg_Soil_Temp_(F)': 'Average soil temperature (°F)'
}

summary = []
for col in df.columns:

    # Getting col
    col_series = df[col]

    # Getting col properties
    dtype = str(col_series.dtype)
    distinct = int(col_series.nunique(dropna=True))
    missing = int(col_series.isnull().sum())

    # Showing examples
    examples = list(pd.Series(col_series.dropna().unique()).astype(str)[:3])
    examples_str = "<><>".join(examples) if examples else ""
    description = description_map.get(col, "NO DESCRIPTION FOUND")

    # Building the list
    summary.append({
        'column': col,
        'dtype': dtype,
        'distinct_values': distinct,
        'missing': missing,
        'examples': examples_str,
        'description': description
    })

# Converting list to DataFrame
summary_df = pd.DataFrame(summary)

# Numeric columns
numeric_cols = df.select_dtypes(include=['number'])
numeric_cols_list = numeric_cols.columns.tolist()
print(f"Numeric columns ({len(numeric_cols_list)}): {numeric_cols_list}")

# Non-numeric columns  
non_numeric_cols = df.select_dtypes(exclude=['number'])
non_numeric_cols_list = non_numeric_cols.columns.tolist()
print(f"Non-numeric columns ({len(non_numeric_cols_list)}): {non_numeric_cols_list}")

# Displaying DataFrame
summary_df

In [ ]:
df.describe()

In [ ]:
# Compute correlation
corr = df.select_dtypes(include=["number"]).corr()

plt.figure(figsize=(8, 10))
sns.heatmap(corr, annot=True, cmap='viridis', fmt='.1f')
plt.title('Correlation matrix (numeric features)')
plt.tight_layout()
plt.show()

In [ ]:
# Distribution of target variable
plt.figure(figsize=(8, 5))
target_counts = df["Target"].value_counts()
plt.bar(['No Fire', 'Fire'], target_counts.values, color=['green', 'red'], alpha=0.7)
plt.title('Distribution of Target Variable (Fire Occurrence)', fontsize=14, fontweight='bold')
plt.ylabel('Count')
plt.xlabel('Fire Status')
for i, v in enumerate(target_counts.values):
    plt.text(i, v + 1000, str(v), ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

print(f"No Fire (0): {target_counts[0]} ({target_counts[0]/len(df["Target"])*100:.2f}%)")
print(f"Fire (1): {target_counts[1]} ({target_counts[1]/len(df["Target"])*100:.2f}%)")

In [ ]:
# Time series: Convert date and analyze temporal patterns
df['Date_parsed'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Year'] = df['Date_parsed'].dt.year
df['Month'] = df['Date_parsed'].dt.month

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Fire occurrences by year
yearly_fires = df.groupby('Year')['Target'].agg(['sum', 'count'])
yearly_fires['fire_rate'] = (yearly_fires['sum'] / yearly_fires['count'] * 100)

axes[0].bar(yearly_fires.index, yearly_fires['sum'], color='darkred', alpha=0.7, label='Fire Count')
ax2 = axes[0].twinx()
ax2.plot(yearly_fires.index, yearly_fires['fire_rate'], color='orange', marker='o', linewidth=2, label='Fire Rate (%)')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Number of Fires', color='darkred')
ax2.set_ylabel('Fire Rate (%)', color='orange')
axes[0].set_title('Fire Occurrences by Year', fontsize=12, fontweight='bold')
axes[0].tick_params(axis='y', labelcolor='darkred')
ax2.tick_params(axis='y', labelcolor='orange')
axes[0].grid(alpha=0.3)

# Fire occurrences by month
monthly_fires = df.groupby('Month')['Target'].agg(['sum', 'count'])
monthly_fires['fire_rate'] = (monthly_fires['sum'] / monthly_fires['count'] * 100)
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

axes[1].bar(range(1, 13), monthly_fires['sum'], color='firebrick', alpha=0.7)
axes[1].set_xticks(range(1, 13))
axes[1].set_xticklabels(month_names)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Number of Fires')
axes[1].set_title('Fire Occurrences by Month', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Non-numeric features: CIMIS Regions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Region distribution
region_counts = df['CIMIS_Region'].value_counts()
axes[0].bar(range(len(region_counts)), region_counts.values, color='mediumseagreen', alpha=0.7)
axes[0].set_xticks(range(len(region_counts)))
axes[0].set_xticklabels(region_counts.index, rotation=45, ha='right', fontsize=9)
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of CIMIS Regions', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3, axis='y')

# Fire occurrence by region
region_fire = df.groupby('CIMIS_Region')['Target'].agg(['sum', 'count'])
region_fire['fire_rate'] = (region_fire['sum'] / region_fire['count'] * 100)
region_fire = region_fire.sort_values('fire_rate', ascending=False)

axes[1].bar(range(len(region_fire)), region_fire['fire_rate'].values, color='orangered', alpha=0.7)
axes[1].set_xticks(range(len(region_fire)))
axes[1].set_xticklabels(region_fire.index, rotation=45, ha='right', fontsize=9)
axes[1].set_ylabel('Fire Occurrence Rate (%)')
axes[1].set_title('Fire Occurrence Rate by CIMIS Region', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Non-numeric features: Station Names
plt.figure(figsize=(12, 6))
station_counts = df['Stn_Name'].value_counts().head(20)
plt.barh(range(len(station_counts)), station_counts.values, color='coral', alpha=0.7)
plt.yticks(range(len(station_counts)), station_counts.index, fontsize=9)
plt.xlabel('Number of Observations')
plt.title('Top 20 Weather Stations by Number of Observations', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Box plots for numeric features by target
fig, axes = plt.subplots(5, 3, figsize=(15, 18))
axes = axes.ravel()

# Exclude 'Target' from features to plot since we're grouping by Target
numeric_features = [col for col in numeric_cols.columns.tolist() if col != 'Target']

for idx, col in enumerate(numeric_features):
    data_to_plot = [df[df['Target'] == 0][col].dropna(), df[df['Target'] == 1][col].dropna()]
    axes[idx].boxplot(data_to_plot, tick_labels=['No Fire', 'Fire'])
    axes[idx].set_title(col, fontsize=10, fontweight='bold')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of numeric features
fig, axes = plt.subplots(5, 3, figsize=(15, 18))
axes = axes.ravel()

# Exclude 'Target' from features to plot since we're grouping by Target
numeric_features = [col for col in numeric_cols.columns.tolist() if col != 'Target']

for idx, col in enumerate(numeric_features):
    axes[idx].hist(df[col].dropna(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[idx].set_title(col, fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Find features with high correlation (> 0.5 or < -0.5)
# Get correlation matrix
corr_matrix = numeric_cols.corr()

# Find pairs with high correlation (excluding diagonal)
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.5:
            high_corr_pairs.append({
                'feature1': corr_matrix.columns[i],
                'feature2': corr_matrix.columns[j],
                'correlation': corr_matrix.iloc[i, j]
            })

# Display high correlation pairs
high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('correlation', ascending=False, key=abs)
print(f"Found {len(high_corr_df)} feature pairs with |correlation| > 0.5:\n")
high_corr_df

In [ ]:
# Create detailed pairwise plots for top highly correlated pairs
num_pairs = len(high_corr_pairs)
if num_pairs > 0:
    top_n = min(6, num_pairs)  # Show top 6 pairs
    top_pairs = high_corr_df.head(top_n)
    
    fig, axes = plt.subplots(top_n, 3, figsize=(15, 4*top_n))
    if top_n == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (_, pair) in enumerate(top_pairs.iterrows()):
        feat1 = pair['feature1']
        feat2 = pair['feature2']
        corr_val = pair['correlation']
        
        # Scatter plot with regression line
        valid_data = df[[feat1, feat2]].dropna()
        axes[idx, 0].scatter(valid_data[feat1], valid_data[feat2], alpha=0.3, s=10)
        z = np.polyfit(valid_data[feat1], valid_data[feat2], 1)
        p = np.poly1d(z)
        x_line = np.linspace(valid_data[feat1].min(), valid_data[feat1].max(), 100)
        axes[idx, 0].plot(x_line, p(x_line), "r--", linewidth=2, label=f'y={z[0]:.2f}x+{z[1]:.2f}')
        axes[idx, 0].set_xlabel(feat1, fontsize=9)
        axes[idx, 0].set_ylabel(feat2, fontsize=9)
        axes[idx, 0].set_title(f'Scatter: {feat1} vs {feat2}', fontsize=10, fontweight='bold')
        axes[idx, 0].legend()
        axes[idx, 0].grid(alpha=0.3)
        
        # Joint distribution by target
        no_fire = df[df['Target'] == 0]
        fire = df[df['Target'] == 1]
        axes[idx, 1].scatter(no_fire[feat1], no_fire[feat2], alpha=0.3, s=10, label='No Fire', c='green')
        axes[idx, 1].scatter(fire[feat1], fire[feat2], alpha=0.3, s=10, label='Fire', c='red')
        axes[idx, 1].set_xlabel(feat1, fontsize=9)
        axes[idx, 1].set_ylabel(feat2, fontsize=9)
        axes[idx, 1].set_title(f'By Target (Corr: {corr_val:.3f})', fontsize=10, fontweight='bold')
        axes[idx, 1].legend()
        axes[idx, 1].grid(alpha=0.3)
        
        # Hexbin plot for density
        hexbin = axes[idx, 2].hexbin(valid_data[feat1], valid_data[feat2], gridsize=30, cmap='YlOrRd', mincnt=1)
        axes[idx, 2].set_xlabel(feat1, fontsize=9)
        axes[idx, 2].set_ylabel(feat2, fontsize=9)
        axes[idx, 2].set_title(f'Density Plot', fontsize=10, fontweight='bold')
        plt.colorbar(hexbin, ax=axes[idx, 2])
    
    plt.tight_layout()
    plt.show()

### 3. Data cleaning:

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
# Removing unnecessary features
df.drop(columns="Stn_Name", inplace=True)

# Parsing date to date object
df['Date_parsed'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

### 4. Feature Engineering

In [ ]:
# Sort by region, station and date to ensure proper time series calculations
df = df.sort_values(['CIMIS_Region', 'Stn_Id', 'Date_parsed']).reset_index(drop=True)

print("Creating rolling average features based on Date...")
print(f"Original dataframe shape: {df.shape}")

# Group by region and station for proper rolling calculations
grouped = df.groupby(['CIMIS_Region', 'Stn_Id'])

# ETo_(in) - previous day, 3 days, 5 days, 1 week, 2 weeks, 3 weeks
for days in [1, 3, 5, 7, 14, 21]:
    col_name = f'ETo_(in)_avg_{days}d'
    df[col_name] = grouped['ETo_(in)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Precip_(in) - previous day, 3 days, 5 days, 1 week, 2 weeks, 3 weeks
for days in [1, 3, 5, 7, 14, 21]:
    col_name = f'Precip_(in)_avg_{days}d'
    df[col_name] = grouped['Precip_(in)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Sol_Rad_(Ly/day) - previous day, 3 days, 5 days
for days in [1, 3, 5]:
    col_name = f'Sol_Rad_(Ly/day)_avg_{days}d'
    df[col_name] = grouped['Sol_Rad_(Ly/day)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Avg_Vap_Pres_(mBars) - previous day, 3 days, 5 days
for days in [1, 3, 5]:
    col_name = f'Avg_Vap_Pres_(mBars)_avg_{days}d'
    df[col_name] = grouped['Avg_Vap_Pres_(mBars)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Max_Air_Temp_(F) - 3 days, 5 days
for days in [3, 5]:
    col_name = f'Max_Air_Temp_(F)_avg_{days}d'
    df[col_name] = grouped['Max_Air_Temp_(F)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Min_Air_Temp_(F) - 3 days, 5 days
for days in [3, 5]:
    col_name = f'Min_Air_Temp_(F)_avg_{days}d'
    df[col_name] = grouped['Min_Air_Temp_(F)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Avg_Air_Temp_(F) - 3 days, 5 days
for days in [3, 5]:
    col_name = f'Avg_Air_Temp_(F)_avg_{days}d'
    df[col_name] = grouped['Avg_Air_Temp_(F)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Max_Rel_Hum_(%) - 3 days, 5 days, 1 week, 2 weeks, 3 weeks
for days in [3, 5, 7, 14, 21]:
    col_name = f'Max_Rel_Hum_(%)_avg_{days}d'
    df[col_name] = grouped['Max_Rel_Hum_(%)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Min_Rel_Hum_(%) - 3 days, 5 days, 1 week, 2 weeks, 3 weeks
for days in [3, 5, 7, 14, 21]:
    col_name = f'Min_Rel_Hum_(%)_avg_{days}d'
    df[col_name] = grouped['Min_Rel_Hum_(%)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Avg_Rel_Hum_(%) - 3 days, 5 days, 1 week, 2 weeks, 3 weeks
for days in [3, 5, 7, 14, 21]:
    col_name = f'Avg_Rel_Hum_(%)_avg_{days}d'
    df[col_name] = grouped['Avg_Rel_Hum_(%)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

# Avg_Soil_Temp_(F) - 3 days, 5 days
for days in [3, 5]:
    col_name = f'Avg_Soil_Temp_(F)_avg_{days}d'
    df[col_name] = grouped['Avg_Soil_Temp_(F)'].transform(lambda x: x.shift(1).rolling(window=days).mean())
    print(f"Created: {col_name}")

print(f"\nFinal dataframe shape: {df.shape}")
print(f"Total new features created: {df.shape[1] - 19}")  # 19 was the original number of columns

In [ ]:
# Show sample data with some new features
sample_cols = ['Date', 'CIMIS_Region', 'Stn_Id', 'ETo_(in)', 'ETo_(in)_avg_1d', 'ETo_(in)_avg_3d', 'ETo_(in)_avg_7d']
print(f"\nSample data with new features:")
df[sample_cols].head(10)

In [ ]:
# Display sample of new features
new_feature_cols = [col for col in df.columns if '_avg_' in col]

# Check for missing values in new features
new_feature_missing = df[new_feature_cols].isnull().sum()
print("Missing values in new rolling average features:")
print(new_feature_missing[new_feature_missing > 0])

# Visualize one example: ETo rolling averages
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot for a single station
station_sample = df[df['Stn_Id'] == df['Stn_Id'].iloc[0]].head(60)

axes[0].plot(station_sample['Date_parsed'], station_sample['ETo_(in)'], label='Original', marker='o', markersize=3)
axes[0].plot(station_sample['Date_parsed'], station_sample['ETo_(in)_avg_1d'], label='1-day avg', alpha=0.7)
axes[0].plot(station_sample['Date_parsed'], station_sample['ETo_(in)_avg_3d'], label='3-day avg', alpha=0.7)
axes[0].plot(station_sample['Date_parsed'], station_sample['ETo_(in)_avg_7d'], label='7-day avg', alpha=0.7)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('ETo (in)')
axes[0].set_title('Example: ETo Rolling Averages (First 60 days of first station)', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot for temperature averages
axes[1].plot(station_sample['Date_parsed'], station_sample['Avg_Air_Temp_(F)'], label='Original', marker='o', markersize=3)
axes[1].plot(station_sample['Date_parsed'], station_sample['Avg_Air_Temp_(F)_avg_3d'], label='3-day avg', alpha=0.7)
axes[1].plot(station_sample['Date_parsed'], station_sample['Avg_Air_Temp_(F)_avg_5d'], label='5-day avg', alpha=0.7)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Avg Air Temp (°F)')
axes[1].set_title('Example: Temperature Rolling Averages', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

Checking if parameters were correcly added

In [ ]:
# There is an error when creatin parameters FOR SURE

df[["CIMIS_Region", "Stn_Id", "Date", "ETo_(in)_avg_1d", "ETo_(in)_avg_21d"]].sort_values(by="ETo_(in)_avg_1d", na_position='first')[:10]

In [ ]:
df[["CIMIS_Region", "Stn_Id", "Date", "ETo_(in)_avg_1d", "ETo_(in)_avg_21d"]].loc[df["Stn_Id"] == 245][20:23]

In [ ]:
# Getting first and last date after removing missing values
print("First date before removing missing values:", df["Date_parsed"].head(1).to_string(index=False))
print("Last date before removing missing values:", df["Date_parsed"].tail(1).to_string(index=False))
print("\nMissing values total:", df.isna().sum().sum())

In [ ]:
# UNDERSTAND WHY I AM DROPPING TIME SERIES AND THE DATE IS NOT CHANING, I AM DOING SOMETHING WRONG
df.dropna(inplace=True)

In [ ]:
# Getting first and last date after removing missing values
print("First date after removing missing values:", df["Date_parsed"].head(1).to_string(index=False))
print("Last date after removing missing values:", df["Date_parsed"].tail(1).to_string(index=False))
print("\nMissing values total:", df.isna().sum().sum())

In [ ]:
# Converting "CIMIS_Region" to numbers using ordinal encoding
encoder = OrdinalEncoder()
encoded = encoder.fit_transform(df[["CIMIS_Region"]])
df["CIMIS_Region_encoded"] = encoded  

print("Ordinal encoding applied to CIMIS_Region:")
for i, region in enumerate(encoder.categories_[0]):
    print(f"  {region}: {i}")
print(f"\nNew column 'CIMIS_Region_encoded' created with shape: {df['CIMIS_Region_encoded'].shape}")

### 5. Model training

In [ ]:
# Separating data and dropping not useful columns
# Remove Target, date-related columns, and original CIMIS_Region (keep encoded version)
X = df.drop(columns=["Target", "Date", "Date_parsed", "Year", "Month", "CIMIS_Region"])
y = df["Target"]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns (first 10): {list(X.columns)[:10]}")
print(f"\nTarget distribution:\n{y.value_counts()}")

# Verify all columns are numeric
print(f"\nData types in X:")
print(X.dtypes.value_counts())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

### Neural Network Training Note

The Neural Network with 50 fully connected layers (100 neurons each) is now integrated with the other models below. It will be trained together with Random Forest, AdaBoost, and Gradient Boosting for comprehensive comparison.

**Neural Network Architecture:**
- 50 hidden layers
- 100 neurons per layer
- ReLU activation
- Adam optimizer
- Early stopping enabled

In [ ]:
# Initialize results dictionary and scoring metrics
results = {}

# Define scoring metrics for cross-validation
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

print("Ready to train models with 3-fold cross-validation")
print("="*80)

In [ ]:
# Model 1: Random Forest
print("\n" + "="*80)
print("RANDOM FOREST")
print("="*80)

# Initialize model with hyperparameters
rf_model = RandomForestClassifier(
    n_estimators=200,      # Number of trees
    random_state=42,
    n_jobs=-1,             # Use all CPU cores
    max_depth=None,        # Unlimited depth
    min_samples_split=2,
    min_samples_leaf=1
)

# Perform cross-validation
print("\nPerforming 3-fold cross-validation...")
cv_results = cross_validate(rf_model, X_train, y_train, cv=3, scoring=scoring, n_jobs=-1)

# Store CV results
results['Random Forest'] = {
    'cv_accuracy': cv_results['test_accuracy'],
    'cv_precision': cv_results['test_precision'],
    'cv_recall': cv_results['test_recall'],
    'cv_f1': cv_results['test_f1']
}

# Print CV results
print(f"\nCross-Validation Results:")
print(f"  Accuracy:  {cv_results['test_accuracy'].mean():.4f} (+/- {cv_results['test_accuracy'].std():.4f})")
print(f"  Precision: {cv_results['test_precision'].mean():.4f} (+/- {cv_results['test_precision'].std():.4f})")
print(f"  Recall:    {cv_results['test_recall'].mean():.4f} (+/- {cv_results['test_recall'].std():.4f})")
print(f"  F1-Score:  {cv_results['test_f1'].mean():.4f} (+/- {cv_results['test_f1'].std():.4f})")

# Train on full training set
print(f"\nTraining on full training set...")
rf_model.fit(X_train, y_train)

# Predict and evaluate on test set
y_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

# Store test results
results['Random Forest']['test_accuracy'] = test_accuracy
results['Random Forest']['test_precision'] = test_precision
results['Random Forest']['test_recall'] = test_recall
results['Random Forest']['test_f1'] = test_f1
results['Random Forest']['model'] = rf_model

# Print test results
print(f"\nTest Set Results:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")

In [ ]:
# Model 2: AdaBoost
print("\n" + "="*80)
print("ADABOOST")
print("="*80)

# Initialize model with hyperparameters
ada_model = AdaBoostClassifier(
    n_estimators=100,      # Number of boosting stages
    random_state=42,
    learning_rate=1.0,     # Weight applied to each classifier
    algorithm='SAMME'      # Algorithm type
)

# Perform cross-validation
print("\nPerforming 3-fold cross-validation...")
cv_results = cross_validate(ada_model, X_train, y_train, cv=3, scoring=scoring, n_jobs=-1)

# Store CV results
results['AdaBoost'] = {
    'cv_accuracy': cv_results['test_accuracy'],
    'cv_precision': cv_results['test_precision'],
    'cv_recall': cv_results['test_recall'],
    'cv_f1': cv_results['test_f1']
}

# Print CV results
print(f"\nCross-Validation Results:")
print(f"  Accuracy:  {cv_results['test_accuracy'].mean():.4f} (+/- {cv_results['test_accuracy'].std():.4f})")
print(f"  Precision: {cv_results['test_precision'].mean():.4f} (+/- {cv_results['test_precision'].std():.4f})")
print(f"  Recall:    {cv_results['test_recall'].mean():.4f} (+/- {cv_results['test_recall'].std():.4f})")
print(f"  F1-Score:  {cv_results['test_f1'].mean():.4f} (+/- {cv_results['test_f1'].std():.4f})")

# Train on full training set
print(f"\nTraining on full training set...")
ada_model.fit(X_train, y_train)

# Predict and evaluate on test set
y_pred = ada_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

# Store test results
results['AdaBoost']['test_accuracy'] = test_accuracy
results['AdaBoost']['test_precision'] = test_precision
results['AdaBoost']['test_recall'] = test_recall
results['AdaBoost']['test_f1'] = test_f1
results['AdaBoost']['model'] = ada_model

# Print test results
print(f"\nTest Set Results:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")

In [ ]:
# Model 3: Gradient Boosting
print("\n" + "="*80)
print("GRADIENT BOOSTING")
print("="*80)

# Initialize model with hyperparameters
gb_model = GradientBoostingClassifier(
    n_estimators=100,      # Number of boosting stages
    random_state=42,
    learning_rate=0.1,     # Shrinks contribution of each tree
    max_depth=3,           # Maximum depth of individual trees
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=1.0          # Fraction of samples for fitting trees
)

# Perform cross-validation
print("\nPerforming 3-fold cross-validation...")
cv_results = cross_validate(gb_model, X_train, y_train, cv=3, scoring=scoring, n_jobs=-1)

# Store CV results
results['Gradient Boosting'] = {
    'cv_accuracy': cv_results['test_accuracy'],
    'cv_precision': cv_results['test_precision'],
    'cv_recall': cv_results['test_recall'],
    'cv_f1': cv_results['test_f1']
}

# Print CV results
print(f"\nCross-Validation Results:")
print(f"  Accuracy:  {cv_results['test_accuracy'].mean():.4f} (+/- {cv_results['test_accuracy'].std():.4f})")
print(f"  Precision: {cv_results['test_precision'].mean():.4f} (+/- {cv_results['test_precision'].std():.4f})")
print(f"  Recall:    {cv_results['test_recall'].mean():.4f} (+/- {cv_results['test_recall'].std():.4f})")
print(f"  F1-Score:  {cv_results['test_f1'].mean():.4f} (+/- {cv_results['test_f1'].std():.4f})")

# Train on full training set
print(f"\nTraining on full training set...")
gb_model.fit(X_train, y_train)

# Predict and evaluate on test set
y_pred = gb_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

# Store test results
results['Gradient Boosting']['test_accuracy'] = test_accuracy
results['Gradient Boosting']['test_precision'] = test_precision
results['Gradient Boosting']['test_recall'] = test_recall
results['Gradient Boosting']['test_f1'] = test_f1
results['Gradient Boosting']['model'] = gb_model

# Print test results
print(f"\nTest Set Results:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")

In [ ]:
# Model 4: Neural Network (50 layers)
print("\n" + "="*80)
print("NEURAL NETWORK (50 LAYERS)")
print("="*80)

# Initialize model with hyperparameters
nn_model = MLPClassifier(
    hidden_layer_sizes=tuple([100] * 90),  # 90 hidden layers with 100 neurons each
    activation='relu',                      # Activation function
    solver='adam',                          # Optimizer
    max_iter=200,                           # Maximum iterations
    random_state=42,
    verbose=True,                           # Show training progress
    early_stopping=True,                    # Stop when validation score stops improving
    validation_fraction=0.1,                # Fraction of training data for validation
    learning_rate_init=0.001,               # Initial learning rate
    batch_size='auto',                      # Batch size for training
    alpha=0.0001                            # L2 regularization parameter
)

# Perform cross-validation
print("\nPerforming 3-fold cross-validation...")
cv_results = cross_validate(nn_model, X_train, y_train, cv=3, scoring=scoring, n_jobs=-1)

# Store CV results
results['Neural Network (50 layers)'] = {
    'cv_accuracy': cv_results['test_accuracy'],
    'cv_precision': cv_results['test_precision'],
    'cv_recall': cv_results['test_recall'],
    'cv_f1': cv_results['test_f1']
}

# Print CV results
print(f"\nCross-Validation Results:")
print(f"  Accuracy:  {cv_results['test_accuracy'].mean():.4f} (+/- {cv_results['test_accuracy'].std():.4f})")
print(f"  Precision: {cv_results['test_precision'].mean():.4f} (+/- {cv_results['test_precision'].std():.4f})")
print(f"  Recall:    {cv_results['test_recall'].mean():.4f} (+/- {cv_results['test_recall'].std():.4f})")
print(f"  F1-Score:  {cv_results['test_f1'].mean():.4f} (+/- {cv_results['test_f1'].std():.4f})")

# Train on full training set
print(f"\nTraining on full training set...")
nn_model.fit(X_train, y_train)

# Predict and evaluate on test set
y_pred = nn_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

# Store test results
results['Neural Network (50 layers)']['test_accuracy'] = test_accuracy
results['Neural Network (50 layers)']['test_precision'] = test_precision
results['Neural Network (50 layers)']['test_recall'] = test_recall
results['Neural Network (50 layers)']['test_f1'] = test_f1
results['Neural Network (50 layers)']['model'] = nn_model

# Print test results
print(f"\nTest Set Results:")
print(f"  Accuracy:  {test_accuracy:.4f}")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall:    {test_recall:.4f}")
print(f"  F1-Score:  {test_f1:.4f}")

In [ ]:
# Summary: All models trained
print("\n" + "="*80)
print("TRAINING COMPLETED FOR ALL 4 MODELS!")
print("="*80)
print(f"\nModels trained: {list(results.keys())}")
print("\nYou can now run the comparison and visualization cells below.")

In [ ]:
# Create comprehensive comparison DataFrame for all 4 models
comparison_data = []
model_names = list(results.keys())

for model_name in model_names:
    comparison_data.append({
        'Model': model_name,
        'CV Accuracy (mean)': results[model_name]['cv_accuracy'].mean(),
        'CV Precision (mean)': results[model_name]['cv_precision'].mean(),
        'CV Recall (mean)': results[model_name]['cv_recall'].mean(),
        'CV F1-Score (mean)': results[model_name]['cv_f1'].mean(),
        'Test Accuracy': results[model_name]['test_accuracy'],
        'Test Precision': results[model_name]['test_precision'],
        'Test Recall': results[model_name]['test_recall'],
        'Test F1-Score': results[model_name]['test_f1']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nModel Comparison Summary (All 4 Models):")
print("="*80)
comparison_df

In [ ]:
# Visualize model performance comparison (All 4 Models)
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metric_keys = ['accuracy', 'precision', 'recall', 'f1']  # Correct metric keys for results dict
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Added 4th color for neural network

model_names = list(results.keys())

for idx, (metric, metric_key) in enumerate(zip(metrics, metric_keys)):
    ax = axes[idx // 2, idx % 2]
    
    # Get CV and Test scores using correct metric keys
    cv_scores = [results[model][f'cv_{metric_key}'].mean() for model in model_names]
    test_scores = [results[model][f'test_{metric_key}'] for model in model_names]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, cv_scores, width, label='CV Mean', alpha=0.8, color=colors)
    bars2 = ax.bar(x + width/2, test_scores, width, label='Test', alpha=0.8, color=[c for c in colors])
    
    # Add error bars for CV scores
    cv_stds = [results[model][f'cv_{metric_key}'].std() for model in model_names]
    ax.errorbar(x - width/2, cv_scores, yerr=cv_stds, fmt='none', ecolor='black', capsize=5, alpha=0.6)
    
    ax.set_xlabel('Model', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=15, ha='right')
    ax.legend()
    ax.grid(alpha=0.3, axis='y')
    ax.set_ylim([0, 1.05])
    
    # Add value labels on bars
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Display confusion matrices for all 4 models
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

for idx, (model_name, model_data) in enumerate(results.items()):
    model = model_data['model']
    y_pred = model.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix in 2x2 grid
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, 
                cbar_kws={'label': 'Count'})
    ax.set_title(f'{model_name}\nConfusion Matrix', fontsize=12, fontweight='bold')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    ax.set_xticklabels(['No Fire', 'Fire'])
    ax.set_yticklabels(['No Fire', 'Fire'])

plt.tight_layout()
plt.show()

In [ ]:
# Detailed classification reports
print("Detailed Classification Reports")
print("="*80)

for model_name, model_data in results.items():
    model = model_data['model']
    y_pred = model.predict(X_test)
    
    print(f"\n{model_name}")
    print("-"*80)
    print(classification_report(y_test, y_pred, target_names=['No Fire', 'Fire'], digits=4))

In [ ]:
# Additional Visualization 1: Side-by-side metric comparison
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metric_keys = ['accuracy', 'precision', 'recall', 'f1']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

model_names = list(results.keys())

for idx, (metric, metric_key) in enumerate(zip(metrics, metric_keys)):
    test_scores = [results[model][f'test_{metric_key}'] for model in model_names]
    
    bars = axes[idx].bar(range(len(model_names)), test_scores, color=colors, alpha=0.8)
    axes[idx].set_title(f'Test {metric}', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel('Score', fontsize=12)
    axes[idx].set_xticks(range(len(model_names)))
    axes[idx].set_xticklabels([name.replace(' (50 layers)', '\n(50 layers)') for name in model_names], 
                                rotation=0, ha='center', fontsize=9)
    axes[idx].set_ylim([0, 1.05])
    axes[idx].grid(alpha=0.3, axis='y')
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Additional Visualization 2: Radar chart comparing all models
fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))

categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
num_vars = len(categories)

# Compute angle for each axis
angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

# Plot data for each model
for idx, (model_name, model_data) in enumerate(results.items()):
    values = [
        model_data['test_accuracy'],
        model_data['test_precision'],
        model_data['test_recall'],
        model_data['test_f1']
    ]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[idx])
    ax.fill(angles, values, alpha=0.15, color=colors[idx])

# Set category labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12, fontweight='bold')

# Set y-axis limits and labels
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], size=10)
ax.grid(True, linestyle='--', alpha=0.7)

plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
plt.title('Model Performance Comparison - Radar Chart', size=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Additional Visualization 3: Heatmap of all metrics
fig, ax = plt.subplots(figsize=(12, 6))

# Prepare data for heatmap
heatmap_data = []
model_names = []
for model_name, model_data in results.items():
    model_names.append(model_name)
    heatmap_data.append([
        model_data['test_accuracy'],
        model_data['test_precision'],
        model_data['test_recall'],
        model_data['test_f1']
    ])

heatmap_df = pd.DataFrame(heatmap_data, 
                            columns=['Accuracy', 'Precision', 'Recall', 'F1-Score'],
                            index=model_names)

sns.heatmap(heatmap_df, annot=True, fmt='.4f', cmap='RdYlGn', 
            vmin=0.5, vmax=1.0, center=0.75,
            linewidths=2, linecolor='white',
            cbar_kws={'label': 'Score'},
            ax=ax)

ax.set_title('Model Performance Heatmap (Test Set)', fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Models', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Additional Visualization 4: Model ranking
fig, ax = plt.subplots(figsize=(14, 8))

# Calculate average performance across all metrics
model_avg_scores = {}
for model_name, model_data in results.items():
    avg_score = (model_data['test_accuracy'] + 
                model_data['test_precision'] + 
                model_data['test_recall'] + 
                model_data['test_f1']) / 4
    model_avg_scores[model_name] = avg_score

# Sort models by average score
sorted_models = sorted(model_avg_scores.items(), key=lambda x: x[1], reverse=True)
model_names_sorted = [item[0] for item in sorted_models]
avg_scores = [item[1] for item in sorted_models]

# Create bar chart
bars = ax.barh(range(len(model_names_sorted)), avg_scores, 
                color=['gold', 'silver', '#CD7F32', 'lightcoral'][:len(model_names_sorted)],
                alpha=0.8, edgecolor='black', linewidth=2)

ax.set_yticks(range(len(model_names_sorted)))
ax.set_yticklabels(model_names_sorted, fontsize=12, fontweight='bold')
ax.set_xlabel('Average Score (across all metrics)', fontsize=12, fontweight='bold')
ax.set_title('Model Ranking by Average Performance', fontsize=16, fontweight='bold', pad=15)
ax.set_xlim([0, 1.05])
ax.grid(alpha=0.3, axis='x')

# Add value labels and ranking
for idx, (bar, score) in enumerate(zip(bars, avg_scores)):
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2.,
            f'{score:.4f}', ha='left', va='center', fontsize=11, fontweight='bold')
    ax.text(0.01, bar.get_y() + bar.get_height()/2.,
            f'#{idx+1}', ha='left', va='center', fontsize=12, fontweight='bold', color='white',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("MODEL RANKING")
print("="*80)
for idx, (model_name, score) in enumerate(sorted_models, 1):
    print(f"#{idx}: {model_name:<30} - Average Score: {score:.4f}")